In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from IPython.display import clear_output
import random
import pandas as pd

In [ ]:
train_number=3000

In [ ]:
image_path='drive/MyDrive/Data/ISIC2018_Task3_Training_Input'
image_dir=os.listdir(image_path)
image_dir.sort()
label_path='drive/MyDrive/Data/ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth.csv'
label_data=pd.read_csv(label_path)
label_data=label_data.to_numpy()

labels=[]
count=0
for i in label_data:
  if count>train_number:
    break
  else:
    count+=1
    array=[]
    for j in i:
        array.append(j)
    array.remove(array[0])
    labels.append(array)
    clear_output(wait=True)
    print("Labels: ",count)

images=[]
count=0
for i in image_dir:
  if count>train_number:
    break
  else:
    if i!='ATTRIBUTION.txt':
      count+=1
      image=cv2.imread(os.path.join(image_path,i))
      image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
      image=cv2.resize(image,(256,256))
      images.append(image)
      clear_output(wait=True)
      print("Images: ",count)

count=0
for i in range(len(images)):
  count+=1
  images[i]=images[i]/255
  clear_output(wait=True)
  print("Normalizing Images...",count)
images=np.array(images)
labels=np.array(labels)

clear_output(wait=True)
print("Finished, ",count)

Finished,  3001


In [ ]:
 def create_conv_model(): 
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(7))

  return model

In [ ]:
basic_model=create_conv_model()

basic_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
basic_model.fit(images,labels,epochs=20,verbose=1)

Epoch 1/42
94/94 [==============================] - 261s 3s/step - loss: 0.3342 - accuracy: 0.6671
Epoch 2/42
94/94 [==============================] - 262s 3s/step - loss: 0.2624 - accuracy: 0.6828
Epoch 3/42
94/94 [==============================] - 258s 3s/step - loss: 0.2619 - accuracy: 0.6828
Epoch 4/42
94/94 [==============================] - 255s 3s/step - loss: 0.2599 - accuracy: 0.6828
Epoch 5/42
94/94 [==============================] - 261s 3s/step - loss: 0.2576 - accuracy: 0.6828
Epoch 6/42
94/94 [==============================] - 258s 3s/step - loss: 0.2581 - accuracy: 0.6838
Epoch 7/42
94/94 [==============================] - 256s 3s/step - loss: 0.2577 - accuracy: 0.6834
Epoch 8/42
94/94 [==============================] - 252s 3s/step - loss: 0.2553 - accuracy: 0.6831
Epoch 9/42
94/94 [==============================] - 261s 3s/step - loss: 0.2540 - accuracy: 0.6841
Epoch 10/42
94/94 [==============================] - 260s 3s/step - loss: 0.2533 - accuracy: 0.6851
Epoch 11/

In [ ]:
basic_model.save('drive/MyDrive/Data/BasicConvolutionalModel.h5')

In [ ]:
def create_discriminator_model():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 512, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))

In [ ]:
models=[]
for i in range(7):
  model=create_discriminator_model()
  models.append(model)

In [ ]:
def create_discriminator_label(label_1,label_2):
  for i in range(len(label_1)):
    if label_1[i]==1.0:
      num=i
      break
  if label_2[num]==1.0:
    label=1
  else:
    label=0
  return label

def create_discriminator_data(image_1,image_2,label_1,label_2):
  label=create_discriminator_label(label_1,label_2)

  new_image=[]
  for i in range(256):
    row=[]
    for j in range(256):
      row.append(image_1[i][j])
    for j in range(256):
      row.append(image_2[i][j])
    new_image.append(row)
  
  return new_image, label

def create_discriminator_dataset(number_samples):
  discriminator_images=[]
  discriminator_labels=[]

  for i in range(number_samples):
    rand_1=random.randint(0,(number_samples-1))
    rand_2=random.randint(0,(number_samples-1))
    image,label=create_discriminator_data(images[rand_1],images[rand_2],labels[rand_1],labels[rand_2])
    discriminator_images.append(image)
    discriminator_labels.append(label)

  return discriminator_images, discriminator_labels